In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
import joblib

In [ ]:
import numpy as np
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt


class SteganographyException(Exception):
    pass

class LSBSteg():
    def __init__(self, im):
        self.image = im
        self.height, self.width, self.nbchannels = im.shape
        self.size = self.width * self.height

        self.maskONEValues = [1,2,4,8,16,32,64,128]
        #Mask used to put one ex:1->00000001, 2->00000010 .. associated with OR bitwise
        self.maskONE = self.maskONEValues.pop(0) #Will be used to do bitwise operations

        self.maskZEROValues = [254,253,251,247,239,223,191,127]
        #Mak used to put zero ex:254->11111110, 253->11111101 .. associated with AND bitwise
        self.maskZERO = self.maskZEROValues.pop(0)

        self.curwidth = 0  # Current width position
        self.curheight = 0 # Current height position
        self.curchan = 0   # Current channel position

    def put_binary_value(self, bits): #Put the bits in the image
        for c in bits:
            val = list(self.image[self.curheight,self.curwidth]) #Get the pixel value as a list
            if int(c) == 1:
                val[self.curchan] = int(val[self.curchan]) | self.maskONE #OR with maskONE
            else:
                val[self.curchan] = int(val[self.curchan]) & self.maskZERO #AND with maskZERO
            self.image[self.curheight,self.curwidth] = tuple(val)
            self.next_slot() #Move "cursor" to the next space

    def next_slot(self):#Move to the next slot were information can be taken or put
        if self.curchan == self.nbchannels-1: #Next Space is the following channel
            self.curchan = 0
            if self.curwidth == self.width-1: #Or the first channel of the next pixel of the same line
                self.curwidth = 0
                if self.curheight == self.height-1:#Or the first channel of the first pixel of the next line
                    self.curheight = 0
                    if self.maskONE == 128: #Mask 1000000, so the last mask
                        raise SteganographyException("No available slot remaining (image filled)")
                    else: #Or instead of using the first bit start using the second and so on..
                        self.maskONE = self.maskONEValues.pop(0)
                        self.maskZERO = self.maskZEROValues.pop(0)
                else:
                    self.curheight +=1
            else:
                self.curwidth +=1
        else:
            self.curchan +=1

    def next_slot2(self): # Hey, we thought this could optimize el time
        self.curchan += 1  # Move to the next channel
        if self.curchan == self.nbchannels:  # If all channels are exhausted
            self.curchan = 0  # Reset channel to the first one
            self.curwidth += 1  # Move to the next pixel on the same line
            if self.curwidth == self.width:  # If all pixels in the row are exhausted
                self.curwidth = 0  # Reset pixel position to the beginning of the row
                self.curheight += 1  # Move to the next row
                if self.curheight == self.height:  # If all rows are exhausted
                    self.curheight = 0  # Reset row position to the beginning
                    if self.maskONE == 128:  # If the last mask is reached
                        raise SteganographyException("No available slot remaining (image filled)")
                    else:  # Use the next available mask
                        self.maskONE = self.maskONEValues.pop(0)
                        self.maskZERO = self.maskZEROValues.pop(0)

    def read_bit(self): #Read a single bit int the image
        val = self.image[self.curheight,self.curwidth][self.curchan]
        # why & self.maskONE
        # val = 0b1011100
        # maskONE = 0b1000000
        # val & maskONE = 0b1000000
        val = int(val) & self.maskONE
        self.next_slot()
        if val > 0:
            return "1"
        else:
            return "0"

    def read_byte(self):
        return self.read_bits(8)

    def read_bits(self, nb): #Read the given number of bits
        bits = ""
        for i in range(nb):
            bits += self.read_bit()
        return bits

    def byteValue(self, val):
        return self.binary_value(val, 8)


    # test case:
    # # Expected binary value: '0110'
    # Explanation: Binary representation of 6 with bitsize 4.
    #binary_value(6, 4) == '0110'
    ###
    def binary_value(self, val, bitsize): #Return the binary value of an int as a byte
        binval = bin(val)[2:] ## delete 0b
        if len(binval) > bitsize:
            raise SteganographyException("binary value larger than the expected size")
        while len(binval) < bitsize:
            binval = "0"+binval
        return binval

    ## encode text by steps:
    # 1. put len of text
    # 2. put each char of text
    def encode_text(self, txt):
        l = len(txt)
        binl = self.binary_value(l, 16) #Length coded on 2 bytes so the text size can be up to 65536 bytes long
        self.put_binary_value(binl) #Put text length coded on 4 bytes
        for char in txt: #And put all the chars
            c = ord(char)
            self.put_binary_value(self.byteValue(c))
        return self.image

    # 1. read len of text
    # 2. read each char of text
    # 3. return the text
    def decode_text(self):
        ls = self.read_bits(16) #Read the text size in bytes
        l = int(ls,2)
        i = 0
        unhideTxt = ""
        while i < l: #Read all bytes of the text
            tmp = self.read_byte() #So one byte
            i += 1
            unhideTxt += chr(int(tmp,2)) #Every chars concatenated to str
        return unhideTxt

    def encode_image(self, imtohide):
        w = imtohide.width
        h = imtohide.height
        if self.width*self.height*self.nbchannels < w*h*imtohide.channels:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        binw = self.binary_value(w, 16) #Width coded on to byte so width up to 65536
        binh = self.binary_value(h, 16)
        self.put_binary_value(binw) #Put width
        self.put_binary_value(binh) #Put height
        for h in range(imtohide.height): #Iterate the hole image to put every pixel values
            for w in range(imtohide.width):
                for chan in range(imtohide.channels):
                    val = imtohide[h,w][chan]
                    self.put_binary_value(self.byteValue(int(val)))
        return self.image


    def decode_image(self):
        width = int(self.read_bits(16),2) #Read 16bits and convert it in int
        height = int(self.read_bits(16),2)
        unhideimg = np.zeros((width,height, 3), np.uint8) #Create an image in which we will put all the pixels read
        for h in range(height):
            for w in range(width):
                for chan in range(unhideimg.channels):
                    val = list(unhideimg[h,w])
                    val[chan] = int(self.read_byte(),2) #Read the value
                    unhideimg[h,w] = tuple(val)
        return unhideimg

    def encode_binary(self, data):
        l = len(data)
        if self.width*self.height*self.nbchannels < l+64:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        self.put_binary_value(self.binary_value(l, 64))
        for byte in data:
            byte = byte if isinstance(byte, int) else ord(byte) # Compat py2/py3
            self.put_binary_value(self.byteValue(byte))
        return self.image

    def decode_binary(self):
        l = int(self.read_bits(64), 2)
        output = b""
        for i in range(l):
            output += chr(int(self.read_byte(),2)).encode("utf-8")
        return output

def encode(image: np.ndarray, message: str) -> np.array:
    # steg = LSBSteg(cv2.imread(filepath))
    steg = LSBSteg(image)
    img_encoded = steg.encode_text(message)
    return img_encoded

def decode(encoded: np.array) -> str:
    steg = LSBSteg(encoded)
    return steg.decode_text()

In [ ]:
import tensorflow as tf
import warnings
# Disabling Future Warnings0
warnings.filterwarnings(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split

import numpy as np

import matplotlib.pyplot as plt

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Hacktrick/models/30_epochs.h5')

In [ ]:
fake = np.load('/content/drive/MyDrive/Hacktrick/Footprints Datasets/fake.npz')
real = np.load('/content/drive/MyDrive/Hacktrick/Footprints Datasets/real.npz')
specto = np.concatenate((fake['x'], real['x']), axis=0).astype(np.float64)  # Change dtype to float32
labels = np.concatenate(([0 for i in range(750)], [1 for i in range(750)]), axis=0)

clip_min = -1e10
clip_max = 1e10
specto = np.clip(specto, clip_min, clip_max)

mean = np.mean(specto, axis=(0, 1))  # Compute mean across samples and time steps
std = np.std(specto, axis=(0, 1))    # Compute standard deviation across samples and time steps

# Step 2: Normalize each feature

combined = list(zip(specto, labels))
# Shuffle the combined list
random.shuffle(combined)
# Unpack the shuffled list back into separate lists
shuffled_specto, shuffled_labels = zip(*combined)

In [ ]:
import requests
import numpy as np

api_base_url = "http://3.70.97.142:5000"
# api_base_url = "http://localhost:3005"
team_id="Lu2xdzj"
# team_id = "xxx"

In [ ]:
def init_eagle(team_id):
    '''
    In this fucntion you need to hit to the endpoint to start the game as an eagle with your team id.
    If a sucessful response is returned, you will recive back the first footprints.
    '''
    payload_sent = {
        'teamId': team_id
    }
    response = requests.post(api_base_url+"/eagle/start", json=payload_sent)
    print(response)
    if response.status_code == 200 or response.status_code == 201:
        print("Game started successfully")
        data = response.json()
        footprints = data['footprint']
        return footprints
    else:
        print("error: ", response.status_code)
    return 0

In [ ]:
idx1 = 98
idx2 = 34
idx3 = 198

In [ ]:
image1 = real['x'][idx1]
image2 = real['x'][idx2]
image3 = fake['x'][idx3]

label1 = 0
label2 = 1
label3 = 0

In [ ]:
# footprints = {
#     '1' : shuffled_specto[idx1],
#     '2' : shuffled_specto[idx2],
#     '3' : shuffled_specto[idx3]
# }
footprints = {
    '1' : image1,
    '2' : image2,
    '3' : image3
}

In [ ]:
def eagle_solve(spect):
  spect = np.expand_dims(spect, axis=0)
  prediction = model.predict(spect)
  return prediction[0]

In [ ]:
def select_channel(footprint):
    '''
    According to the footprint you recieved (one footprint per channel)
    you need to decide if you want to listen to any of the 3 channels or just skip this message.
    Your goal is to try to catch all the real messages and skip the fake and the empty ones.
    Refer to the documentation of the Footprints to know more what the footprints represent to guide you in your approach.
    '''

    '''
    foorprint = {
    '1': spect,
    }
    for i in footpring:
        call model (footprint)
        if return == 1 return channel id

    return 0
    '''
    predictions = {}
    for channel, spect in footprint.items():
        spect = np.array(spect).astype(np.float64)

        spect = np.clip(spect, clip_min, clip_max)

        mean2 = np.mean(spect, axis=(0, 1))

        if abs(mean2) < 0.5:
          predictions[channel] = 0
          continue

        normalized_spect = (spect - mean) / std

        spect = np.expand_dims(normalized_spect, axis=0)
        prediction = model.predict(spect)[0]

        if prediction > 0.5:
            return int(channel)


    return 0

In [ ]:
# c = select_channel(footprints)
# print(c)
# print('ddddddddddddddddddddddddddddd')
# print(shuffled_labels[idx1])
# print(shuffled_labels[idx2])
# print(shuffled_labels[idx3])

c = select_channel(footprints)
print(c, type(c))
print('ddddddddddddddddddddddddddddd')
print(label1)
print(label2)
print(label3)

1/1 [==============================] - 0s 110ms/step
1 <class 'int'>
ddddddddddddddddddddddddddddd
0
1
0


In [ ]:
def skip_msg(team_id):
    '''
    If you decide to NOT listen to ANY of the 3 channels then you need to hit the end point skipping the message.
    If sucessful request to the end point , you will expect to have back new footprints IF ANY.
    '''
    payload_sent = {
        'teamId': team_id
    }
    response = requests.post(api_base_url+"/eagle/skip-message", json=payload_sent)
    print(response)
    if response.status_code == 200 or response.status_code == 201:
        print("Message Skipped")
        if response.text == "End of message reached":
            print(response.text)

        data = response.json()
        footprints = data['nextFootprint']
        return footprints
    else:
        if response.text == "End of message reached":
          print(response.text, "status: ", response.status_code)
        else:
          print("error: ", response.status_code)
    return 0

In [ ]:
def request_msg(team_id, channel_id):
    '''
    If you decide to listen to any of the 3 channels then you need to hit the end point of selecting a channel to hear on (1,2 or 3)
    '''
    payload_sent = {
        'teamId': team_id,
        'channelId': channel_id
    }
    response = requests.post(api_base_url+"/eagle/request-message", json=payload_sent)
    print(response)
    if response.status_code == 200 or response.status_code == 201:
        print("Message Request")
        data = response.json()
        encodedMsg = data['encodedMsg']
        return encodedMsg
    else:
        print("error: ", response.status_code)
    return 0

In [ ]:
def submit_msg(team_id, decoded_msg):
    '''
    In this function you are expected to:
        1. Decode the message you requested previously
        2. call the api end point to send your decoded message
    If sucessful request to the end point , you will expect to have back new footprints IF ANY.
    '''
    payload_sent = {
        'teamId': team_id,
        'decodedMsg': decoded_msg
    }
    response = requests.post(api_base_url+"/eagle/submit-message", json=payload_sent)
    print(response)
    if response.status_code == 200 or response.status_code == 201:
        print("Message Submit")
        if response.text == "End of message reached":
          print(response.text)

        data = response.json()
        footprints = data['nextFootprint']
        return footprints
    else:
        if response.text == "End of message reached":
          print(response.text, "status: ", response.status_code)
        else:
          print("error: ", response.status_code)
    return 0

In [ ]:
def end_eagle(team_id):
    '''
    Use this function to call the api end point of ending the eagle  game.
    Note that:
    1. Not calling this fucntion will cost you in the scoring function
    '''
    payload_sent = {
        'teamId': team_id,
    }
    response = requests.post(api_base_url+"/eagle/end-game", json=payload_sent)
    print(response)
    if response.status_code == 200 or response.status_code == 201:
        print("Game Ends ", response.text)
        return response
    else:
        print("error: ", response.status_code)
    return 0

In [ ]:
footprints = init_eagle(team_id)
while 1:
  if footprints == 0:
      end_eagle(team_id)
      break

  channel_id = select_channel(footprints)
  print(channel_id)
  if channel_id == 0:
      footprints = skip_msg(team_id)
      continue
  else:
      image_msg = request_msg(team_id, channel_id)
      image_array = np.array(image_msg,dtype=np.uint8)
      text = decode(image_array)
      print(text)
      footprints = submit_msg(team_id, text)
      continue

<Response [201]>
Game started successfully
1/1 [==============================] - 2s 2s/step
{'1': array([8.001587e-05], dtype=float32), '2': 0, '3': 0}
0
<Response [200]>
Message Skipped
1/1 [==============================] - 0s 29ms/step
{'1': array([0.8994018], dtype=float32), '2': 0, '3': array([0.00027775], dtype=float32)}
1
<Response [200]>
Message Request
Cryptograp
<Response [200]>
Message Submit
1/1 [==============================] - 0s 28ms/step
{'1': array([0.95111793], dtype=float32), '2': array([0.00027775], dtype=float32), '3': array([3.2072556e-05], dtype=float32)}
1
<Response [200]>
Message Request
hy is cool
<Response [400]>
End of message reached status:  400
<Response [200]>
Game Ends  Game ended successfully with a score of 75.07307802008367. New Highscore reached!


In [ ]:
print(image_msg)

[[[12, 10, 32], [14, 14, 36], [12, 12, 32], [12, 16, 36], [21, 26, 49], [22, 32, 57], [27, 36, 67], [34, 44, 80], [50, 63, 107], [49, 59, 104], [54, 63, 108], [42, 47, 84], [34, 40, 70], [36, 42, 61], [23, 24, 37], [16, 20, 31], [20, 23, 41], [19, 24, 44], [19, 25, 44], [20, 25, 44]], [[10, 8, 30], [12, 14, 35], [13, 14, 34], [10, 17, 35], [16, 25, 43], [18, 31, 53], [29, 37, 68], [41, 51, 86], [45, 57, 99], [43, 52, 99], [51, 60, 103], [41, 48, 84], [29, 34, 63], [23, 28, 48], [17, 21, 32], [26, 30, 39], [18, 24, 40], [18, 23, 43], [19, 24, 44], [20, 25, 45]], [[10, 9, 27], [13, 14, 32], [14, 15, 33], [11, 17, 33], [16, 23, 41], [18, 29, 49], [30, 39, 68], [46, 56, 92], [57, 69, 111], [49, 60, 105], [51, 60, 103], [40, 48, 85], [26, 33, 61], [16, 23, 42], [11, 20, 29], [25, 32, 40], [19, 25, 41], [19, 24, 43], [20, 25, 44], [21, 26, 45]], [[14, 13, 29], [16, 15, 31], [13, 15, 30], [11, 17, 31], [16, 23, 39], [20, 28, 49], [32, 41, 70], [49, 59, 94], [74, 86, 126], [59, 70, 115], [45, 

In [ ]:
print(channel_id, team_id)

1 Lu2xdzj


In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/Hacktrick/models/30epochs.h5', '/content/')

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/Hacktrick/Footprints Datasets/real.npz', '/content/')
shutil.copy('/content/drive/MyDrive/Hacktrick/Footprints Datasets/fake.npz', '/content/')

In [15]:
!python main.py

2024-03-05 21:58:52.932740: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 21:58:52.932795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 21:58:52.934657: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 21:58:52.945598: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-05 21:58:54.542478: W tensorflow/comp

In [ ]:
array_of_imgs = np.load('/content/hanyy.npy')

In [ ]:
from PIL import Image

for i, img_array in enumerate(array_of_imgs):
    img = Image.fromarray(img_array)
    img.save(f"/content/image_{i}.png")